In [6]:
import xarray
import numpy as np
import pandas
import h5py
import datetime

In [65]:
# SST_L3S_VIIRS table
# this contains instructions for what cutouts to create: it matches the VIIRS to L3S

l3s = pandas.read_parquet('/Users/rachelstumpf/SST_L3S_VIIRS.parquet')

In [3]:
# tmp.parquet
# this is the temporary table that contains extracted info about each cutout

tmp = pandas.read_parquet('/Users/rachelstumpf/ulmo/ulmo/runs/L3S/vet/tmp.parquet')

In [4]:
# tst.h5
# this is the extraction files. 'valid' is the temp arrays, 'valid_metadata' is the associated things to keep track of

file_path = '/Users/rachelstumpf/tst.h5'
with h5py.File(file_path, 'r') as h5_file:
    dataset = h5_file['valid']
    first_observation = dataset[0]

In [51]:
# keep only dates that are before or on 02-02-2012

gd_date = l3s.datetime <= datetime.datetime(2012,2,2)
l3s_table = l3s[gd_date]

l3s_table['VIIRS_datetime']

0      2012-02-01 00:10:00
1      2012-02-01 00:10:00
2      2012-02-01 00:10:00
3      2012-02-01 00:10:00
4      2012-02-01 00:10:00
               ...        
1809   2012-02-02 23:40:00
1810   2012-02-02 23:40:00
1811   2012-02-02 23:40:00
1812   2012-02-02 23:40:00
1813   2012-02-02 23:40:00
Name: VIIRS_datetime, Length: 1364, dtype: datetime64[ns]

In [36]:
l3s_table['datetime']

0      2012-01-31 13:42:26
1      2012-01-31 13:44:47
2      2012-01-31 13:49:19
3      2012-01-31 13:46:48
4      2012-01-31 13:44:00
               ...        
1809   2012-02-01 14:24:56
1810   2012-02-01 14:20:56
1811   2012-02-01 14:23:53
1812   2012-02-01 14:19:50
1813   2012-02-01 13:52:44
Name: datetime, Length: 1364, dtype: datetime64[ns]

In [21]:
l3s_table['ex_filename'][1365]

'/Volumes/Aqua-1/Hackathon/daily/l3s_fields/2012/033/20120202120000-STAR-L3S_GHRSST-SSTsubskin-LEO_Daily-ACSPO_V2.80-v02.0-fv01.0.nc'

In [34]:
l3s_table

,VIIRS_row,VIIRS_col,VIIRS_UID,VIIRS_LL,VIIRS_pp_file,VIIRS_pp_idx,VIIRS_T90,VIIRS_T10,VIIRS_DT,VIIRS_pp_type,...,VIIRS_filename,VIIRS_ex_filename,VIIRS_lat,VIIRS_lon,row,col,lat,lon,datetime,ex_filename
0,3024,2784,13281093372310754,481.689453,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,32205,26.920013,26.429993,0.490021,0,...,s3://viirs/data/2012/032/20120201001000-OSPO-L...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,19.337208,-176.89246,3533,155,19.337208,-176.89246,2012-01-31 13:42:26,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...
1,3120,2688,13281101161369470,360.780640,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,252131,26.670013,25.809998,0.860016,0,...,s3://viirs/data/2012/032/20120201001000-OSPO-L...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,20.116112,-176.30530,3494,184,20.116112,-176.30530,2012-01-31 13:44:47,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...
2,3216,2496,13281110096482976,496.087891,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,459126,26.359985,25.950012,0.409973,0,...,s3://viirs/data/2012/032/20120201001000-OSPO-L...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,21.009575,-175.17024,3449,241,21.009575,-175.17024,2012-01-31 13:49:19,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...
3,3216,2592,13281108764419858,454.589905,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,379958,26.309998,25.829987,0.480011,0,...,s3://viirs/data/2012/032/20120201001000-OSPO-L...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,20.876402,-175.80142,3456,209,20.876402,-175.80142,2012-01-31 13:46:48,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...
4,3216,2688,13281107257349902,326.442383,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,217941,26.359985,25.609985,0.750000,0,...,s3://viirs/data/2012/032/20120201001000-OSPO-L...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,20.725744,-176.50098,3463,174,20.725744,-176.50098,2012-01-31 13:44:00,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,2448,1728,132813537581373470,217.640198,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,126440,7.739990,6.880005,0.859985,0,...,s3://viirs/data/2012/033/20120202234000-OSPO-L...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,45.375824,-166.26530,2231,686,45.375824,-166.26530,2012-02-01 14:24:56,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...
1810,2448,1824,132813519631273463,211.481689,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,939851,8.269989,6.899994,1.369995,0,...,s3://viirs/data/2012/033/20120202234000-OSPO-L...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,45.196297,-167.26537,2240,636,45.196297,-167.26537,2012-02-01 14:20:56,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...
1811,2544,1728,132813598841346893,299.721680,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,301365,7.089996,6.679993,0.410004,0,...,s3://viirs/data/2012/033/20120202234000-OSPO-L...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,45.988450,-166.53107,2200,673,45.988450,-166.53107,2012-02-01 14:23:53,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...
1812,2544,1824,132813580671245860,323.458008,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,688847,7.429993,6.690002,0.739990,0,...,s3://viirs/data/2012/033/20120202234000-OSPO-L...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,45.806660,-167.54140,2209,622,45.806660,-167.54140,2012-02-01 14:19:50,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...


In [39]:
l3s_table.iloc[[0,1363]]['VIIRS_datetime']

0      2012-02-01 00:10:00
1813   2012-02-02 23:40:00
Name: VIIRS_datetime, dtype: datetime64[ns]

In [40]:
l3s_table.iloc[[0,1363]]['datetime']

0      2012-01-31 13:42:26
1813   2012-02-01 13:52:44
Name: datetime, dtype: datetime64[ns]

In [41]:
l3s_table.iloc[[0,1363]]['lon']

0      -176.89246
1813   -174.31564
Name: lon, dtype: float64

In [47]:
# oh nevermind this is okay because they were all subtracted from 1:30am. So if we were setting
# each L3S cutout to be at 1:30 am, a lot of them would be shifted to an earlier day. I don't understand
# why we have no cutouts from 02-02-2012, unless they're out of order

l3s_table.sort_values('VIIRS_datetime')[['VIIRS_datetime', 'lon', 'datetime']]

# this makes sense, very low longitudes will be shifted earlier the most
# so there must just be a handicap with keeping observations on 02-02, since a lot of longitudes shift it to 02-01


,VIIRS_datetime,lon,datetime
0,2012-02-01 00:10:00,-176.89246,2012-01-31 13:42:26
18,2012-02-01 00:10:00,-169.23978,2012-01-31 14:13:02
17,2012-02-01 00:10:00,-169.06831,2012-01-31 14:13:44
16,2012-02-01 00:10:00,-168.31488,2012-01-31 14:16:44
15,2012-02-01 00:10:00,-168.89798,2012-01-31 14:14:24
...,...,...,...
1787,2012-02-02 23:40:00,-172.35194,2012-02-01 14:00:36
1786,2012-02-02 23:40:00,-172.05122,2012-02-01 14:01:48
1812,2012-02-02 23:40:00,-167.54140,2012-02-01 14:19:50
1797,2012-02-02 23:40:00,-175.37512,2012-02-01 13:48:30


In [49]:
# so why are there 1364 rows, but indexes from 0 to 1813??

l3s_table.sort_values(['lon','VIIRS_datetime'])[['VIIRS_datetime', 'lon', 'datetime']]

,VIIRS_datetime,lon,datetime
1329,2012-02-02 14:00:00,-179.84679,2012-02-01 13:30:37
387,2012-02-01 14:20:00,-179.83559,2012-01-31 13:30:39
28,2012-02-01 01:50:00,-179.58327,2012-01-31 13:31:40
29,2012-02-01 01:50:00,-178.89117,2012-01-31 13:34:26
31,2012-02-01 01:50:00,-177.80217,2012-01-31 13:38:47
...,...,...,...
389,2012-02-01 14:20:00,169.14713,2012-02-01 12:46:35
391,2012-02-01 14:20:00,169.74101,2012-02-01 12:48:58
390,2012-02-01 14:20:00,169.90172,2012-02-01 12:49:36
388,2012-02-01 14:20:00,170.06294,2012-02-01 12:50:15


In [80]:
l3s['date'] = pandas.to_datetime(l3s['datetime'])
l3s['date'] = l3s['date'].dt.date
l3s['date']

# Create a list of target dates
target_dates = pd.to_datetime(['2012-01-31', '2012-02-01', '2012-02-02']).date

# Select rows where the date is in the target_dates list
selected_rows = l3s[l3s['date'].isin(target_dates)]


In [81]:
l3s['date']

0          2012-01-31
1          2012-01-31
2          2012-01-31
3          2012-01-31
4          2012-01-31
              ...    
2932447    2020-12-30
2932448    2020-12-30
2932449    2020-12-30
2932450    2020-12-31
2932451    2020-12-31
Name: date, Length: 2932452, dtype: object

In [82]:
target_dates

array([datetime.date(2012, 1, 31), datetime.date(2012, 2, 1),
       datetime.date(2012, 2, 2)], dtype=object)

In [83]:
selected_rows

,VIIRS_row,VIIRS_col,VIIRS_UID,VIIRS_LL,VIIRS_pp_file,VIIRS_pp_idx,VIIRS_T90,VIIRS_T10,VIIRS_DT,VIIRS_pp_type,...,VIIRS_ex_filename,VIIRS_lat,VIIRS_lon,row,col,lat,lon,datetime,ex_filename,date
0,3024,2784,13281093372310754,481.689453,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,32205,26.920013,26.429993,0.490021,0,...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,19.337208,-176.89246,3533,155,19.337208,-176.89246,2012-01-31 13:42:26,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...,2012-01-31
1,3120,2688,13281101161369470,360.780640,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,252131,26.670013,25.809998,0.860016,0,...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,20.116112,-176.30530,3494,184,20.116112,-176.30530,2012-01-31 13:44:47,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...,2012-01-31
2,3216,2496,13281110096482976,496.087891,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,459126,26.359985,25.950012,0.409973,0,...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,21.009575,-175.17024,3449,241,21.009575,-175.17024,2012-01-31 13:49:19,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...,2012-01-31
3,3216,2592,13281108764419858,454.589905,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,379958,26.309998,25.829987,0.480011,0,...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,20.876402,-175.80142,3456,209,20.876402,-175.80142,2012-01-31 13:46:48,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...,2012-01-31
4,3216,2688,13281107257349902,326.442383,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,217941,26.359985,25.609985,0.750000,0,...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,20.725744,-176.50098,3463,174,20.725744,-176.50098,2012-01-31 13:44:00,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...,2012-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2834,1872,1632,132812846252208366,327.829895,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,410308,13.500000,12.600006,0.899994,0,...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,38.462463,-157.91634,2576,1104,38.462463,-157.91634,2012-02-02 14:58:20,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...,2012-02-02
2835,1968,1440,132812934552357419,177.556030,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,223771,12.769989,11.239990,1.529999,0,...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,39.345470,-156.42581,2532,1178,39.345470,-156.42581,2012-02-02 15:04:18,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...,2012-02-02
2836,1968,1536,132812921712272424,238.436462,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,443943,12.839996,11.769989,1.070007,0,...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,39.217148,-157.27576,2539,1136,39.217148,-157.27576,2012-02-02 15:00:54,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...,2012-02-02
2837,1968,1632,132812908172187222,254.114441,s3://viirs/PreProc/VIIRS_2012_95clear_192x192_...,136820,13.149994,12.279999,0.869995,0,...,s3://viirs/Extractions/VIIRS_2012_95clear_192x...,39.081715,-158.12778,2545,1093,39.081715,-158.12778,2012-02-02 14:57:29,/Volumes/Aqua-1/Hackathon/daily/l3s_fields/201...,2012-02-02
